# Introduction to ConvNets (CNN) in keras

Now that you have some idea of how to use keras, we will use that to build a convolutional neural network! It is the main type of network that we will utilize to program our self-driving cars.


# The problem/dataset

As our toy problem, we will look into CIFAR10, this is one of a good, publicly open dataset that can be used to test your model for general object classification performance. In this dataset contains 60K images in 32x32 resolution. Not only does this have a good amount of data for proper learning to take place, it has multiple classes to practice multi-class classification task.

Citation: Learning Multiple Layers of Features from Tiny Images, Alex Krizhevsky, 2009.

In [0]:
#imports
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras import applications
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
import tensorflow.keras.backend as K
from glob import glob
from tensorflow.keras.datasets import cifar10

# Dataset

We will first load our dataset. The process is as follows:

## Mac/Linux-based OS
1. Download the python version for CIFAR-10 from [here](https://www.cs.toronto.edu/~kriz/cifar.html)
2. Uncompress the .tar.gz file

## Windows
1. Download the python version for CIFAR-10 from [here](https://www.cs.toronto.edu/~kriz/cifar.html)
2. 


In [2]:
from tensorflow.keras.utils import to_categorical
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

num_classes = 10

# Convert class vectors to binary class matrices.
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

#scale all our data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

from sklearn.model_selection import train_test_split

#Since test data is separetly provided, we do split for train/val
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train,test_size=0.4, random_state=42)
print(x_train.shape, x_val.shape, y_train.shape, y_val.shape)



170500096/170498071 [==============================] - 47s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
(30000, 32, 32, 3) (20000, 32, 32, 3) (30000, 10) (20000, 10)


# Model (from scratch)

In [0]:
def build_model():
    input_layer = Input(shape=(32,32,3))
    conv1 = Conv2D(32, (3,3), strides=1, activation='relu', padding = 'same') (input_layer)
    pool1 = MaxPooling2D((2,2))(conv1)
    conv2 = Conv2D(64, (3,3), strides=1, activation='relu', padding = 'same') (pool1)
    pool2 = MaxPooling2D((2,2))(conv2)
    conv3 = Conv2D(128, (3,3), strides=1, activation='relu', padding = 'same') (pool2)
    pool3 = MaxPooling2D((2,2))(conv3)
    conv4 = Conv2D(256, (3,3), strides=1, activation='relu', padding = 'same') (pool3)
    glob = GlobalMaxPooling2D() (conv4)
    fc1 = Dense(512) (glob)
    fc1 = Dropout(rate= 0.4) (fc1)
    fc2 = Dense(1024) (fc1)
    fc2 = Dropout(rate= 0.4)(fc2)
    fc3 = Dense(10, activation = 'softmax') (fc2)
    model = Model(input_layer, fc3)
    model.summary()
    return model

# Model (from keras.applications)

In [0]:
def call_xception():
    model = applications.xception.Xception(include_top=True, weights=None, input_tensor=Input((32,32,3)), pooling='max', classes=10)
    model.compile(optimizer = Adam(), loss = 'categorical_crossentropy', metrics = ['acc'])
    model.summary()
    return model

# Compile, train, and test

In [0]:
def train_and_eval(type = 'custom'):
    if (type=='custom'):
        model = build_model()
        model.compile(optimizer = Adam(), loss = 'categorical_crossentropy', metrics = ['acc'])
        model.fit(x_train, y_train, batch_size = 128, epochs = 2, shuffle = True, validation_data=(x_val,y_val),
                  verbose = 1)
        model.evaluate(x_test, y_test, verbose = 1)
    if (type=='xception'):
        model = call_xception()
        model.fit(x_train, y_train, batch_size = 16, epochs = 30, shuffle = True, validation_data=(x_val,y_val),
                  verbose = 1)
        model.evaluate(x_test, y_test, verbose = 1)

In [6]:
train_and_eval()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)    

# Graphs!

Congratulations! You now used various CNN models for the image classification task with Keras.